# imports

In [1]:
import pickle
from utils import split_validation
from srgnn_pl import SRGNN_model, SRGNN_Map_Dataset
import torch
import os

from torch.utils.data import DataLoader
import pytorch_lightning as pl
import wandb
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import torch.utils.data as data_utils
import numpy as np

In [2]:
class fake_parser():
    def __init__(self,
                 dataset='yoochoose1_64',
                 batchSize=128,
                 hiddenSize=128,
                 epoch=30,
                 lr=1e-3,
                 lr_dc=0.1,
                 lr_dc_step=3,
                 l2=1e-5,
                 step=1,
                 patience=10,
                 nonhybrid=False,
                 validation=True,
                 valid_portion=0.1,
                 pretrained_embedings=False):
        self.dataset=dataset
        self.batchSize=batchSize
        self.hiddenSize=hiddenSize
        self.epoch=epoch
        self.lr=lr
        self.lr_dc=lr_dc
        self.lr_dc_step=lr_dc_step
        self.l2=l2
        self.step=step
        self.patience=patience
        self.nonhybrid=nonhybrid
        self.validation=validation
        self.valid_portion=valid_portion
        self.pretrained_embedings=pretrained_embedings

In [3]:
opt=fake_parser(step=3, batchSize=128, hiddenSize=256, epoch=60, validation=True, valid_portion=0.2,
                nonhybrid=True,
               dataset='yoochoose_custom_augmented_5050')

In [4]:
train_data = pickle.load(open('../datasets/' + opt.dataset  + '/train.txt', 'rb'))
if opt.validation:
    train_data, valid_data = split_validation(train_data, opt.valid_portion)
else:
    test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))

if opt.dataset == 'diginetica':
    n_node = 43098
elif opt.dataset == 'yoochoose1_64' or opt.dataset == 'yoochoose1_4':
    n_node = 37484
elif opt.dataset == 'yoochoose_custom':
    n_node = 28583
elif opt.dataset == 'yoochoose_custom_augmented':
    n_node = 27809
elif opt.dataset == 'yoochoose_custom_augmented_5050':
    n_node = 27807
else:
    n_node = 310


In [52]:
class SRGNN_sampler(data_utils.Sampler):
    def __init__(self, dataset, batch_size, shuffle=False, drop_last=False):
        self.dataset=dataset
        self.batch_size=batch_size
        self.shuffle=shuffle
        self.drop_last=drop_last

    def __len__(self):
        return self.dataset.length
    
    def __iter__(self):
        order=np.arange(len(self))
        if self.shuffle:
            np.random.shuffle(order)
        if len(self)%self.batch_size:
            for i in range(0, len(self)-self.batch_size, self.batch_size):
                yield order[i:i+self.batch_size]
            if not self.drop_last:
                yield order[-(len(self)%self.batch_size):]
        else:
            for i in range(0, len(self), self.batch_size):
                yield order[i:i+self.batch_size]
       # raise IndexError('Done iterating')

In [53]:
train_dataset=SRGNN_Map_Dataset(train_data, shuffle=True)
del train_data
val_dataset=SRGNN_Map_Dataset(valid_data)
del valid_data

train_dataloader=DataLoader(train_dataset, 
                            #batch_size=opt.batchSize, 
                            num_workers=os.cpu_count(),  
                          #  worker_init_fn=worker_init_fn, 
                            sampler=SRGNN_sampler(train_dataset, opt.batchSize, shuffle=True, drop_last=False)
                            #drop_last=
                            )
#del train_dataset
val_dataloader=DataLoader(val_dataset, 
                          #batch_size=opt.batchSize, 
                          num_workers=os.cpu_count(), 
                          sampler=SRGNN_sampler(val_dataset, opt.batchSize, shuffle=False, drop_last=False)

                        #  worker_init_fn=worker_init_fn
                         )
#del val_dataset

data masking start
data masking 1
data masking 2
data masking 3
done masking
data masking start
data masking 1
data masking 2
data masking 3
done masking


In [38]:
model=SRGNN_model(opt, n_node, 
                  init_embeddings=None,
                  **(opt.__dict__))

In [14]:
torch.set_float32_matmul_precision('medium')

In [ ]:
wandb_logger = pl.loggers.WandbLogger(project='GNN_master',entity="kpuchalskixiv",
                                      log_model="all")

In [54]:
trainer=pl.Trainer(max_epochs=60,
                   limit_train_batches=train_dataset.length//opt.batchSize,
                   limit_val_batches=val_dataset.length//opt.batchSize,
                   callbacks=[
                              EarlyStopping(monitor="val_loss", patience=6, mode="min", check_finite=True)],
                   logger=wandb_logger
                  )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [57]:
trainer.fit(model=model, 
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader
            )

/home/kacper/anaconda3/envs/gpu_torch/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /home/kacper/SR-GNN/pytorch_code/lightning_logs/version_15/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type         | Params
---------------------------------------
0 | model | SessionGraph | 10.6 M
---------------------------------------
10.6 M    Trainable params
0         Non-trainable params
10.6 M    Total params
42.593    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Traceback (most recent call last):
  File "/home/kacper/anaconda3/envs/gpu_torch/lib/python3.11/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/kacper/anaconda3/envs/gpu_torch/lib/python3.11/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kacper/anaconda3/envs/gpu_torch/lib/python3.11/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/home/kacper/anaconda3/envs/gpu_torch/lib/python3.11/shutil.py", line 738, in rmtree
    onerror(os.rmdir, path, sys.exc_info())
  File "/home/kacper/anaconda3/envs/gpu_torch/lib/python3.11/shutil.py", line 736, in rmtree
    os.rmdir(path, dir_fd=dir_fd)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-a137fgfd'
/home/kacper/anaconda3/envs/gpu_torch/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The d

Training: 0it [00:00, ?it/s]

In [ ]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
